In [2]:
from gensim.models import word2vec
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from numpy import array
import gensim
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import gensim
import re
import spacy
from scipy.spatial.distance import cosine

In [3]:
import gensim
import re
import spacy

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [4]:
Qu = pd.read_excel('./data/Question.xlsx')
df = pd.read_csv('./data/class.csv')

In [5]:
Qu['سوال']

0               اين قاعة
1             وين القاعه
2        اين اجد القاعه 
3       اين مكان القاعه 
4         اين هي القاعه 
5               مين يدرس
6             مين دكتور 
7            من هو دكتور
8                من يدرس
9              من محاضر 
10       متى تبدا محاضره
11           متى محاضره 
12              متى درس 
13       متى اختبار مادة
14            متى اختبار
15       متى موعد اختبار
16            اقدر انزل 
17       هل استطيع تنزيل
18          هل اقدر انزل
19         هل اقدر اسجل 
20    هل من الممكن تسجيل
Name: سوال, dtype: object

In [11]:
trylist = list(df['المقرر'].unique())

In [24]:
def split_digits_and_words(s):
    result = []
    X = 0
    for c in s:
        if c.isdigit():
            if X ==1 :
                X = 0 
                result.append(" ")
            result.append(int(c))
        else:
            result.append(c)
            X = 1 
            
    s = ''.join(str(x) for x in result)

    return s






'MATH 115'

In [27]:
for i in range(len(trylist)):
    
    trylist[i] = split_digits_and_words(trylist[i])

In [15]:
trylist[0]

'MATH115'

In [7]:
df.head(3)

,فتره,جاهزة,المستفيد,المحاضر,القاعة,الى,من,الايام,المسجلين,اعلى حد,التسلسل,النشاط,ساعات المقرر,اسم المقرر,المقرر,الشعبة,index
0,12,نعم,44,اسلام خان محمد نضرول,304,11:10 ص,09:45 ص,1,37,50,1,نظري,3,حساب التكامل,MATH115,846,1
1,12,نعم,44,اسلام خان محمد نضرول,304,09:15 ص,08:00 ص,4,37,50,1,نظري,3,حساب التكامل,MATH115,846,2
2,12,نعم,44,اسلام خان محمد نضرول,304,09:15 ص,08:00 ص,2 1,43,50,2,نظري,3,حساب التكامل,MATH115,847,3


In [29]:
count = 0
quList = []
for i in Qu['سوال']:
    for j in trylist:
        a = i + " " + j
        quList.append(a)
        print(a)
        count+=1

اين قاعة MATH 115
اين قاعة MATH 116
اين قاعة MATH. 326
اين قاعة PHYS 116
اين قاعة STAT 126
اين قاعة COE 223
اين قاعة COE 351
اين قاعة COE 352
اين قاعة CS 211
اين قاعة CS 213
اين قاعة CS 214
اين قاعة CS 221
اين قاعة CS 222
اين قاعة CS 224
اين قاعة CS 315
اين قاعة CS 341
اين قاعة CS 342
اين قاعة CS 348
اين قاعة CS 361
اين قاعة CS 383
اين قاعة CS 423
اين قاعة CS 432
اين قاعة CS 435
اين قاعة CS 451
اين قاعة CS 452
اين قاعة CS 471
اين قاعة CS 492
اين قاعة CS 497
اين قاعة CS 498
اين قاعة CS 499
اين قاعة IT 351
اين قاعة MATH 212
اين قاعة MATH 314
اين قاعة MATH 317
اين قاعة MATH 329
اين قاعة COE 211
اين قاعة COE 212
اين قاعة COE 213
اين قاعة COE 214
اين قاعة COE 224
اين قاعة COE 231
اين قاعة COE 325
اين قاعة COE 326
اين قاعة COE 332
اين قاعة COE 333
اين قاعة COE 361
اين قاعة COE 362
اين قاعة COE 415
اين قاعة COE 428
اين قاعة COE 432
اين قاعة COE 434
اين قاعة COE 435
اين قاعة COE 441
اين قاعة COE 442
اين قاعة COE 455
اين قاعة COE 457
اين قاعة COE 463
اين قاعة COE 492
اين قاعة COE 493
اين قاعة C

In [30]:
sen1 = quList[0].split() #taking only the first Question in the list 'اين قاعة'
sen1

['اين', 'قاعة', 'MATH', '115']

In [38]:
# mode2 = gensim.models.Word2Vec.load('./data/aravec/GoogleNews-vectors-negative300.bin')

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [64]:
from scipy.spatial.distance import cosine
import nltk

#Load AraVec

model = gensim.models.Word2Vec.load('./data/aravec/wikipedia_cbow_100')
#import Dataset that have synonyms Question about the same thing 
# A = pd.read_excel('./data/Question.xlsx')
#converting the Dataframe into list 
# docs = 
# this Example of Question came from the user it also should include the name of class but this is how it should looks after cleaning 
target = "متى هو الموعد  المخصص لاختبار ماده math 115" 
#spliting the sentence into words

max = 0
max1 = []
for j in range(len(quList)):
    sen1 = quList[j].split() #taking only the first Question in the list 'اين قاعة'
    sen2 = target.split()
    wordVec1 = 0
    for i in range(len(sen1)):
        #cleaning the word
        try:
            word = clean_str(sen1[i])
        # adding the vectors of all words
            wordVec1 += model.wv[ word]
        except:
            continue
        
    wordVec2 = 0
    for i in range(len(sen2)):
        try:
            word = clean_str(sen2[i])
            wordVec2 += model.wv[ word]
        except:
            continue
    #finding the cosine similarity of the two sentences 
    similarity = 1 - cosine(wordVec1, wordVec2)
    if similarity > max :
        max = similarity
        max1 = sen1
    print(similarity)
print( max1)
print( max)

0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956
0.01094294898211956


0.3563941717147827
